In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import urllib2

In [2]:
def getlinks():
    # gets a list of links from top 250 page
    response = urllib2.urlopen('http://www.imdb.com/chart/top?ref_=ft_250')
    content = response.read()
    soup = BeautifulSoup(content, 'html.parser')
    targets = soup.select('.lister-list')[0].find_all('a')
    linkstodo = ['http://www.imdb.com'+re.findall('(?<=<a href=").*(?=\/\?)', str(x))[0] for x in targets][::2]
    return linkstodo

def generatesublink(link):
    # All the demographic sublinks
    sublinks = ['/ratings-all',
               '/ratings-male',
               '/ratings-female',
               '/ratings-age_1',
               '/ratings-male_age_1',
               '/ratings-female_age_1',
               '/ratings-age_2',
               '/ratings-male_age_2',
               '/ratings-female_age_2',
               '/ratings-age_3',
               '/ratings-male_age_3',
               '/ratings-female_age_3',
               '/ratings-age_4',
               '/ratings-male_age_4',
               '/ratings-female_age_4',
               '/ratings-imdb_staff',
               '/ratings-top_1000',
               '/ratings-usa',
               '/ratings-international']
    
    for sublink in sublinks:
        yield link+sublink
                       
def getsoupresult(link):
    response = urllib2.urlopen(link)
    content = response.read()
    soup = BeautifulSoup(content, 'html.parser')
    return soup

def create_series(soupresult):
    # Set the title
    title = re.findall('(?<=>).+(?=</a>)', str(soupresult.find_all('a', class_='main')))
    return pd.Series(title, index=['Title'])

def get_total_stats(soupresult):
    # Pre parsing soupresult
    total_stats = soupresult.select('#tn15content')[0].select('p')
    # Create index for series
    index = ['Total_votes', 'Mean', 'Median', 'N_votes_10', 'N_votes_9', 'N_votes_8','N_votes_7', 
                 'N_votes_6','N_votes_5','N_votes_4','N_votes_3','N_votes_2', 'N_votes1']
    # check if no breakdown is present
    breakdown = re.findall('No breakdown', str(total_stats[0]))
    if len(breakdown) == 0:

        # set the subdemographic for column titles
        subdemo = re.findall('(?<=\d\s).+(?=\shave)', str(total_stats[0]))[0]
        # Total voters and weighted average
        stats = re.findall('(?<=<p>)\d*(?=\s)', str(total_stats[0]))
        # Mean and Median
        stats = stats + (re.findall('(?<==\s)[0-9]\.?[0-9]?', str(total_stats[2])))
        # Number of votes per rating
        stats = stats + re.findall('(?<=>)[0-9]+', str(soupresult.select('#tn15content > table > tr > td')))[::2]
        
        index = map(lambda x: subdemo+'_'+x, index)
        stats = pd.Series(stats, index=index)
        return stats
    else:
        # If there is no breakdown present, just return an empty series
        return pd.Series(index=index)
    
def iterlinks(linkstodo):
    data = []
    # iterate the llist of links
    for movie,link in enumerate(linkstodo):
        # This is only here to set the movie title
        first_soup = getsoupresult(link+'/ratings-all')
        title_data = create_series(first_soup)
        iteratedata = [title_data]
        
        # iterate the sublinks to all the demographics
        for num,sublink in enumerate(generatesublink(link)):
            # Get the soup
            soup = getsoupresult(sublink)
            # turn it in to a serie
            demographicstats = get_total_stats(soup)
            # append it to the datalist of this title
            iteratedata.append(demographicstats)
        # concat all movie stats and append to full data list
        one_movie_frame = pd.concat(iteratedata).to_frame().T
        data.append(one_movie_frame)
        print movie
    # concat full data list and return
    data = pd.concat(data, ignore_index=True)
    
    data.drop(['Mean','Median','N_votes1','N_votes_10','N_votes_2',
                 'N_votes_3','N_votes_4','N_votes_5','N_votes_6','N_votes_7',
                 'N_votes_8','N_votes_9',], axis=1, inplace=True, errors='ignore')
    return data

def main():
    linkstodo = getlinks()
    data = iterlinks(linkstodo)
    return data

In [3]:
data = main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [8]:
data.to_excel('IMDb.xlsx')